In [ ]:
# Initial imports and notebook setup, click arrow to show
import sys 
import os

from HARK.ConsumptionSaving.ConsAggShockModel import *
from HARK.utilities import plotFuncsDer, plotFuncs
import matplotlib.pyplot as plt

This module contains one model to solve a consumption-saving model in which an agent have CRRA preferences over consumption who discounts future utility flows and expects to experience permanent and transitory shocks to his/her income. 
The agent faces idiosyncratic and aggregate productivity shocks to income. That is, s/he believes that the market to which his/her supply (a fixed amount of) labor will experience **aggregate** permanent and transitory shocks to the effective productivity of labor. 

Also, the wage rate in the market is the marginal product of labor in the aggregate production function, and the interest factor is one plus the (net) marginal product of capital. 
One can assume that the ratio of aggregate capital-to-labor is a sufficient statistic for these marginal products. 

Further, the agent believes that the capital-to-labor ratio evolves as a function of its current value. 

### Idiosyncratic and Aggregate Shocks to Income

The agent's problem can be written in (normalized) Bellman form as:

\begin{eqnarray*}
v_t(m_t,k_t) &=& \max_{c_t} u(c_t) + \beta (1-\mathsf{D}_{t+1}) \mathbb{E} [(\Gamma_{t+1}\psi_{t+1})^{1-\rho} v_{t+1}(m_{t+1}, k_{t+1}) ], \\
a_t &=& m_t - c_t, \\
a_t &\geq& 0, \\
m_{t+1} &=& R_{t+1}/(\Gamma_{t+1}\psi_{t+1}\Psi_{t+1}) a_t + W_{t+1}\theta_{t+1}, \\
R_{t+1} &=& \mathbf{R}(k_{t+1}/\Theta_{t+1}),  W_{t+1}= \mathbf{W}(k_{t+1}/\Theta_{t+1}), \\
k_{t+1} &=& \mathbf{k}(k_{t}),\\
\psi_{t} \sim F_{\psi_{t}},&& \quad \theta_{t} \sim F_{\theta_{t}},\quad \mathbb{E} [F_{\psi t}] = 1, \\
\Psi_{t} \sim F_{\Psi_{t}},&& \quad \Theta_{t} \sim F_{\Theta_{t}},\quad \mathbb{E} [F_{\Psi t}] = \mathbb{E}[F_{\Theta t}]=1, \\
u(c) &=& \frac{c^{1-\rho}}{1-\rho}
\end{eqnarray*}

The objects **R**(.) and **W**(.) are functions of the effective capital-to-labor ratio that generate the (net) interest factor and wage rate, respectively. These are determined by the aggregate production function and the degree of capital depreciation. 

The $k(.)$ function represents the agent's belifs about the evolution of the capital-to-labor ratio $k_{t}$.
As with iodiosyncratic shocks, there is an aggregate shock process $(F_{\Theta},F_{\Psi})$.

The one period problem of this model is solved by the function $\texttt{solveConsAggShock}$, the default value of $\texttt{solveOnePeriod}$ for$\texttt{AggShockConsumerType}$.
The class $\texttt{AggShockConsumerType}$ extends $\texttt{IndShockConsumerType}$ to represents agents in this model.
To construct an instance of this class, similar parameters to the ones for $\texttt{IndShockConsumerType}$ must be passed to the constructor. However, several attributes have been *removed*: 
- $\texttt{Rfree}$ is endogenous, 
- $\texttt{CubicBool}$, $\texttt{vFuncBool}$, and $\texttt{BoroCnstArt}$ are not yet supported in this model.

The only **new** attribute is $\texttt{kGridBase}$, an array of scaling factor for the (perfect foresight equivalent) steady state capital ratio. It is used to construct $\texttt{kGrid}$, a direct input for $\texttt{solveConsAggShock}$.

In [ ]:
# This cell defines a dictionary to make an instance of an "aggregate shocks" consumer.
AggShocksDict={
    "CRRA": 2.0,                           # Coefficient of relative risk aversion
    "Rfree": 1.03,                         # Interest factor on assets
    "DiscFac": 0.96,                       # Default intertemporal discount factor
    "LivPrb" : [0.98],                     # Survival probability
    "AgentCount" : 10000,                  # Number of agents of this type (only matters for simulation)
    "PermGroFac" :[1.0],                  # Permanent income growth factor   

    # Parameters for constructing the "assets above minimum" grid
    "aNrmInitMean" : 0.0,                  # Mean of log initial assets (only matters for simulation)
    "aNrmInitStd"  : 0.0,                  # Standard deviation of log initial assets (only for simulation)
    "pLvlInitMean" : 0.0,                  # Mean of log initial permanent income (only matters for simulation)
    "pLvlInitStd"  : 0.0,                  # Standard deviation of log initial permanent income (only matters for simulation)
    #"PermGroFacAgg" : 1.0,              # Aggregate permanent income growth factor (only matters for simulation)
    "T_age" : None,                        # Age after which simulated agents are automatically killed
    "T_cycle" : 1,                         # Number of periods in the cycle for this agent type
    "aXtraMin" : 0.001,                    # Minimum end-of-period "assets above minimum" value
    "aXtraMax" : 20,                       # Maximum end-of-period "assets above minimum" value               
    "aXtraExtra" : [None],                   # Some other value of "assets above minimum" to add to the grid
    "aXtraNestFac" : 3,                    # Exponential nesting factor when constructing "assets above minimum" grid
    "aXtraCount" : 24,                     # Number of points in the grid of "assets above minimum"
    
    # Parameters describing the income process
    "PermShkCount" : 7,                    # Number of points in discrete approximation to permanent income shocks
    "TranShkCount" : 7,                    # Number of points in discrete approximation to transitory income shocks
    "PermShkStd" : [0.1],                  # Standard deviation of log permanent shocks to income
    "TranShkStd" : [0.1],                  # Standard deviation of log transitory shocks to income
    "UnempPrb" : 0.05,                     # Probability of unemployment while working
    "UnempPrbRet" : 0.005,                 # Probability of "unemployment" while retired
    "IncUnemp" : 0.3,                      # Unemployment benefits replacement rate
    "IncUnempRet" : 0.0,                   # "Unemployment" benefits when retired
    "tax_rate" : 0.0,                      # Flat income tax rate
    "T_retire" : 0,                        # Period of retirement (0 --> no retirement)
    
    # A few other paramaters
    "BoroCnstArt" : 0.0,                   # Artificial borrowing constraint; imposed minimum level of end-of period assets
    "CubicBool" : False,                   # Preference shocks currently only compatible with linear cFunc
    "vFuncBool" : False,                    # Whether to calculate the value function during solution 
    
    "MgridBase" : np.array([0.1,0.3,0.6,0.8,0.9,0.98,1.0,1.02,1.1,1.2,1.6,2.0,3.0]),  # Grid of capital-to-labor-ratios (factors)
}
del AggShocksDict['Rfree']         # Interest factor is endogenous in agg shocks model
del AggShocksDict['CubicBool']     # Not supported yet for agg shocks model
del AggShocksDict['vFuncBool']     # Not supported yet for agg shocks model

In [ ]:
# Make an example of an "aggregate shocks"consumer
AggShockExample = AggShockConsumerType(**AggShocksDict)
AggShockExample.cycles = 0 # Make this type have an infinite horizon

In [ ]:
# Solve the microeconomic model for the aggregate shocks example type
AggShockExample.solve()
AggShockExample.unpackcFunc()

In [ ]:
# Plot the results
print('Consumption function at each aggregate market resources-to-labor ratio gridpoint:')
m_grid = np.linspace(0,10,200)
for M in AggShockExample.Mgrid.tolist():
    mMin = AggShockExample.solution[0].mNrmMin(M)
    c_at_this_M = AggShockExample.cFunc[0](m_grid+mMin,M*np.ones_like(m_grid))
    plt.plot(m_grid+mMin,c_at_this_M)
plt.show()

### Cobb-Douglas Economy

A model with “aggregate shocks” only makes sense if there is some market-level object that experiences these shocks. The $\texttt{CobbDouglasEconomy}$ class extends $\texttt{Market}$ to represent an economy with a Cobb-Douglas production function over aggregate capital and aggregate labor and permanent and transitory shocks to labor productivity.

The basic model for the Cobb-Douglas economy is:
\begin{eqnarray*}
Y &=& K^{\alpha}L^{1-\alpha}, \quad k \equiv K/L \\
W = \frac{\delta Y}{\delta L} &=& (1-\alpha)K^{\alpha}L^{-\alpha} = (1-\alpha)k^{\alpha}\\
r  = \frac{\delta Y}{\delta K} &=& \alpha K^{\alpha-1}L^{1-\alpha} = \alpha k^{\alpha-1}\\
R &=& 1+ r - \delta\\
\end{eqnarray*}

### Parameters to solve CobbDouglasEconomy

A new instance of $\texttt{CobbDouglasEconomy}$ must have attributes listed in the table below. 

The constructor for the class uses these attributes to calculate the perfect foresight steady state of the capital-to-labor ratio, wage rate, and interest rate; the interest and wage functions; a discretization of the aggregate shock process, and an initial guess of the next- capital-ratio function. 

Following [Krusell and Smith (1998)](https://www.journals.uchicago.edu/doi/pdf/10.1086/250034), we assume that the log of next period’s capital ratio is a linear function of the log of this period’s capital ratio.

### Example parameter values to solve an instance of CobbDouglasEconomy

| Param | Description | Code | Value | Constructed |
| :---: | --- | --- | --- | :---: |
| $\alpha$ |Capital's share of output  | $\texttt{CapShare}$ | 0.36 | |
| $\delta$ |Capital's depreciation rate  | $\texttt{DeprFac}$ | 0.025 | |
| $\sigma_\theta $ | Standard deviation of log transitory agg shock distribution | $\texttt{TranShkAggStd}$ | - |$\surd$ |
| $ \sigma_\psi $ | Standard deviation of log permanent agg shock distribution  | $\texttt{PermShkaggStd}$ | - |$\surd$ |
| $ N_{\theta} $ | Number of discrete values in transitory agg shock distribution  | $\texttt{TranShkAggCount}$  | 3 | 
| $ N_{\psi} $ | Number of discrete values in permanent agg shock distribution  | $\texttt{PermShkAggCount}$  | 3 | 
| $\beta ^{PF}$ |Perfect foresight intertemporal discount factor  | $\texttt{DiscFac}$ | 0.96 | |
| $\rho ^{PF} $ |Perfect foresight coefficient of relative risk aversion | $\texttt{CRRA}$ | 2.0 | |

In [ ]:
# This cell defines a dictionary to make an instance of a "Cobb-Douglas" consumer.
CobbDouglasDict={
    'PermShkAggCount': 3,           # Number of points in discrete approximation to aggregate permanent shock dist
    'TranShkAggCount': 3,           # Number of points in discrete approximation to aggregate transitory shock dist
    'PermShkAggStd': 0.0063,        # Standard deviation of log aggregate permanent shocks
    'TranShkAggStd': 0.0031,        # Standard deviation of log aggregate transitory shocks
    'DeprFac': 0.025,               # Capital depreciation rate
    'CapShare': 0.36,               # Capital's share of income
    'DiscFac': 0.96,                # Discount factor of perfect foresight calibration
    'CRRA': 2.0,                    # Coefficient of relative risk aversion of perfect foresight calibration
    'PermGroFacAgg': 1.0,           # Aggregate permanent income growth factor (only matters for simulation)
    'AggregateL':1.0,
    'act_T':1200,
    'intercept_prev': 0.0,          # Intercept of aggregate savings function
    'slope_prev': 1.0,              # Slope of aggregate savings function      
}

In [ ]:
# Make a Cobb-Douglas economy for the agents
EconomyExample = CobbDouglasEconomy(agents = [AggShockExample],**CobbDouglasDict)
EconomyExample.makeAggShkHist() # Simulate a history of aggregate shocks

In [ ]:
# Have the consumers inherit relevant objects from the economy
AggShockExample.getEconomyData(EconomyExample)

We can solve for the equilibrium of a Cobb-Douglas economy. This might take a few minutes...

In [ ]:
# Solve the "macroeconomic" model by searching for a "fixed point dynamic rule"
EconomyExample.solve()

In [ ]:
print('Aggregate savings as a function of aggregate market resources:')
plotFuncs(EconomyExample.AFunc,0,2*EconomyExample.kSS)

print('Consumption function at each aggregate market resources gridpoint (in general equilibrium):')
AggShockExample.unpackcFunc()
m_grid = np.linspace(0,10,200)
AggShockExample.unpackcFunc()
for M in AggShockExample.Mgrid.tolist():
    mMin = AggShockExample.solution[0].mNrmMin(M)
    c_at_this_M = AggShockExample.cFunc[0](m_grid+mMin,M*np.ones_like(m_grid))
    plt.plot(m_grid+mMin,c_at_this_M)
plt.show()